In [120]:
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, WordNetLemmatizer
from nltk.probability import FreqDist
from pathlib import Path
import glob
import random
import pandas as pd
import re
import requests
from tqdm.notebook import tqdm
from sklearn.model_selection import StratifiedKFold, cross_val_score, RepeatedStratifiedKFold
from nltk.classify.scikitlearn import SklearnClassifier
from sklearn.naive_bayes import MultinomialNB,BernoulliNB,GaussianNB,ComplementNB
from statistics import mean
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, TfidfTransformer
from sklearn import metrics

In [56]:
txt_files = glob.glob('/Users/antonhesse/Desktop/Anton/Education/UMN/Lab and Research/HSPL/CPET_scoping_review/data/cpet_articles/txts/*.txt')

In [57]:
manual_text_df = pd.read_csv('/Users/antonhesse/Desktop/Anton/Education/UMN/Lab and Research/HSPL/CPET_scoping_review/data/cpet_articles/Manual text analysis - Data.csv')
manual_text_df['txt_file_name'] = manual_text_df.apply(lambda x: str(x['Article'] + '.txt'), axis=1)
manual_text_df = manual_text_df[manual_text_df['Eligible'] == 'e']

In [58]:
analyzed_txt_files = manual_text_df['txt_file_name'].to_list()

In [59]:
def process_file(file_name, file_list):
    txt_re = re.compile(file_name)
    fname = list(filter(txt_re.search, file_list))[0]
    
    # check file size to make sure the txt file actually has text
    file_size = 0
    while file_size == 0:
        file_size = Path(fname).stat().st_size
        if file_size != 0: # check if conversion to txt didn't work
            with open(fname, 'r') as f:
                text = f.read()
        else:
            print('Empty file, returning None')
            return None
    text_lower = text.lower()
    tokens = word_tokenize(text_lower)
    stop_words = set(stopwords.words('english'))

    filtered_tokens = [t for t in tokens if t not in stop_words]
    
    lemmatizer = WordNetLemmatizer()
    lemmatized_words = [lemmatizer.lemmatize(t) for t in filtered_tokens]
    
    return lemmatized_words

In [60]:
def get_raw_text(file_name, file_list):
    txt_re = re.compile(file_name)
    fname = list(filter(txt_re.search, file_list))[0]
    
    # check file size to make sure the txt file actually has text
    file_size = 0
    while file_size == 0:
        file_size = Path(fname).stat().st_size
        if file_size != 0: # check if conversion to txt didn't work
            with open(fname, 'r') as f:
                text = f.read()
        else:
            print('Empty file, returning None')
            return None
    text_lower = text.lower()

In [67]:
words = [process_file(f, txt_files) for f in tqdm(analyzed_txt_files)]
joined_words = [' '.join(text) for text in words]

  0%|          | 0/104 [00:00<?, ?it/s]

In [68]:
gas_data = manual_text_df['Gas data'].to_list()

In [149]:
# text = ['The quick brown fox jumped over the lazy dog.',
#        'The dog.',
#        'The fox.']
from sklearn.model_selection import train_test_split
features_train, features_test, labels_train, labels_test = train_test_split(joined_words, gas_data, test_size=0.25, random_state=10)


In [150]:
vectorizer = TfidfVectorizer(stop_words='english')

In [151]:
features_train = vectorizer.fit_transform(features_train)
features_test = vectorizer.transform(features_test)

In [152]:
# X = vectorizer.fit_transform(joined_words)
# vocabulary = vectorizer.get_feature_names_out()

In [153]:
BNB = BernoulliNB()
BNB.fit(features_train, labels_train)

BernoulliNB()

In [154]:
BNB_pred = BNB.predict(features_test)
accuracy_BNB = metrics.accuracy_score(labels_test, BNB_pred)
print("BernoulliNB accuracy:", accuracy_BNB)
print("confusion matrix:")
print(metrics.confusion_matrix(labels_test, BNB_pred))

BernoulliNB accuracy: 0.6923076923076923
confusion matrix:
[[ 0  8]
 [ 0 18]]


In [155]:
MNB = MultinomialNB()
MNB.fit(features_train, labels_train)

MultinomialNB()

In [156]:
MNB_pred = MNB.predict(features_test)
accuracy_MNB = metrics.accuracy_score(labels_test, MNB_pred)
print("BernoulliNB accuracy:", accuracy_MNB)
print("confusion matrix:")
print(metrics.confusion_matrix(labels_test, MNB_pred))

BernoulliNB accuracy: 0.6923076923076923
confusion matrix:
[[ 0  8]
 [ 0 18]]


In [16]:
# vector = TfidfVectorizer(stop_words='english')
# vector.fit(joined_words)
# vector.get_feature_names_out()
# tfidf = vector.transform(joined_words)

In [17]:
# tfidf_transformer = TfidfTransformer()
# X_train_tfidf = tfidf_transformer.fit_transform

In [28]:
# tfidf = vector.transform(joined_words)

TfidfVectorizer(stop_words='english')

In [161]:
from sklearn.pipeline import Pipeline
text_clf = Pipeline([
    ('vect', CountVectorizer()),
    ('tfidf', TfidfTransformer()),
    ('clf', BernoulliNB()),
])

In [164]:
# text_clf.fit(X = features_train, y = labels_train)

<78x29823 sparse matrix of type '<class 'numpy.float64'>'
	with 112336 stored elements in Compressed Sparse Row format>